In [3]:
#!pip install nlp
#!pip install sklearn

In [ ]:
from nlp import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Training and fine-tuning

Model classes in 🤗 Transformers are designed to be compatible with native PyTorch and TensorFlow 2 and can be used seemlessly with either. We will show how to use our included `Trainer()` class which handles much of the complexity of training for you.

What's included:

* How to use `Trainer()` class which handles much of the complexity of training for you. Based on [this link](https://huggingface.co/transformers/training.html#trainer)
* How to use fast.ai to fine-tune transformers. Based on [this tutorial](https://docs.fast.ai/tutorial.transformers)

What's NOT included:

* Traning using native tensorflow or PyTorch.

## 1. HuggingFace Trainer
We provide a simple but feature-complete training and evaluation interface through `Trainer()` and `TFTrainer()`. You can train, fine-tune, and evaluate any 🤗 Transformers model with a wide range of training options and with built-in features like logging, gradient accumulation, and mixed precision.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# loading the pretrained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

In [7]:
# loading the fine-tuned model
model = BertForSequenceClassification.from_pretrained("/storage/yiping-transformer/results/checkpoint-2000/")

In [8]:
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])

In [9]:
# the dataset is nothing magical.
test_dataset[0]

{'label': 1,
 'text': "I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."}

In [10]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=1024)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=1024)

In [11]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [12]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/storage/yiping-transformer/results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/storage/yiping-transformer/logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

In [ ]:
result = trainer.predict(test_dataset)

In [ ]:
result

In [13]:
trainer.evaluate()

/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


{'eval_loss': 0.272245317697525}

In [16]:
%load_ext tensorboard
%tensorboard --logdir "/storage/yiping-transformer/logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 6523), started 0:05:02 ago. (Use '!kill 6523' to kill it.)